This notebook demonstrates multiple use cases of using the Spotify API to automate playlist creation and edition. 

In [1]:
import spotipy
import os
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from pprint import pprint
import pandas as pd
from dotenv import load_dotenv

Checking that you have your Spotify developer credentials. Get your secret keys on https://developer.spotify.com/

To run examples on this notebook, you can store your credentials on a "dotenv" file (.env) and easily access them using the Dotenv library. Your dotenv file should contain these three lines:

SPOTIPY_CLIENT_ID="your client id"
<br>SPOTIPY_CLIENT_SECRET="your client secret"
<br>SPOTIPY_REDIRECT_URI=http://localhost:8888/callback

In [2]:
'''
dotenv file should contain:
SPOTIPY_CLIENT_ID = "your client id"
SPOTIPY_CLIENT_SECRET = "your client secret"  
SPOTIPY_REDIRECT_URI = "http://localhost:8888/callback"
'''
load_dotenv("C:/apis/.env") # path to your dotenv file
client_id = os.getenv("SPOTIPY_CLIENT_ID")
client_secret = os.getenv("SPOTIPY_CLIENT_SECRET")
redirect_uri = os.getenv("SPOTIPY_REDIRECT_URI")

# This masks your secret keys before printing them, in case you are sharing this notebook:
def mask_secret(unmasked_chars, secret):
    masked_token = secret[:unmasked_chars] + '*' * (len(secret) - unmasked_chars*2) + secret[-unmasked_chars:]
    return masked_token

print(f"SPOTIPY_CLIENT_ID: {mask_secret(4, client_id)}")
print(f"SPOTIPY_CLIENT_SECRET: {mask_secret(4, client_secret)}")
print(f"SPOTIPY_REDIRECT_URI: {redirect_uri}")

SPOTIPY_CLIENT_ID: 3068************************2505
SPOTIPY_CLIENT_SECRET: 9945************************2daa
SPOTIPY_REDIRECT_URI: http://localhost:8888/callback


# Sort a playlist by artist popularity

In [3]:
def sort_playlist_by_artist_popularity(client, playlist_url):
    print("Starting sort_playlist_by_artist_popularity")
    playlist_id = playlist_url.split("/")[-1].split("?")[0]
    print(f"Playlist ID: {playlist_id}")
    tracks_data = []

    try:
        results = client.playlist_items(playlist_id, limit=100)
        tracks = []
        while results:
            tracks.extend(results['items'])
            if results['next']:
                results = client.next(results)
            else:
                results = None
        print(f"Tracks fetched: {len(tracks)}")
    except Exception as e:
        print(f"Error fetching playlist items: {e}")
        return

    for i, track_item in enumerate(tracks):
        try:
            track = track_item['track']
            uri = track['uri']  
            artist_id = track['artists'][0]['id']
            artist = client.artist(artist_id)
            artist_name = artist['name']
            artist_popularity = artist['popularity']
            genres = artist['genres']
            followers = artist['followers']['total']
            track_name = track['name']
            tracks_data.append({
                "Artist name": artist_name,
                "Artist id": artist_id,
                "Track name": track_name,
                "Artist popularity": artist_popularity,
                "Genres": genres,
                "Followers": followers,
                "uri": uri 
            })
            print(f"Processed track {i+1}/{len(tracks)} by {artist_name} - Popularity: {artist_popularity}, Followers: {followers}")
        except Exception as e:
            print(f"Error processing track {i+1}: {e}")

    # Sort tracks_data by artist popularity and followers before creating DataFrame
    sorted_tracks_data = sorted(tracks_data, key=lambda x: (x["Artist popularity"], x["Followers"]), reverse=True)
    df_tracks_sorted = pd.DataFrame(sorted_tracks_data)

    print("Tracks sorted by artist popularity and followers")

    try:
        user_id = client.current_user()["id"]
        print(f"User ID: {user_id}")

        original_playlist = client.playlist(playlist_id)
        original_name = original_playlist['name']
        new_playlist_name = original_name + " - sorted by artist popularity"
        new_playlist = client.user_playlist_create(user_id, new_playlist_name, public=False)
        new_playlist_id = new_playlist["id"]
        track_uris = df_tracks_sorted["uri"].tolist() 
        print(f"Creating new playlist with ID: {new_playlist_id}")

        for i in range(0, len(track_uris), 100):
            client.playlist_add_items(new_playlist_id, track_uris[i:i+100])
            print(f"Added tracks {i+1}-{min(i+100, len(track_uris))} to new playlist")
    except Exception as e:
        print(f"Error during playlist creation or track addition: {e}")

    return df_tracks_sorted

# Sort a playlist by track popularity

In [5]:
def sort_playlist_by_track_popularity(client, playlist_url):
    print("Starting sort_playlist_by_track_popularity")
    playlist_id = playlist_url.split("/")[-1].split("?")[0]
    print(f"Playlist ID: {playlist_id}")
    tracks_data = []

    try:
        results = client.playlist_items(playlist_id, limit=100)
        tracks = []
        while results:
            tracks.extend(results['items'])
            if results['next']:
                results = client.next(results)
            else:
                results = None
        print(f"Tracks fetched: {len(tracks)}")
    except Exception as e:
        print(f"Error fetching playlist items: {e}")
        return

    for i, track_item in enumerate(tracks):
        try:
            track = track_item['track']
            uri = track['uri']  
            artist_id = track['artists'][0]['id']
            artist = client.artist(artist_id)
            artist_name = artist['name']
            track_popularity = track['popularity']
            genres = artist['genres']
            followers = artist['followers']['total']
            track_name = track['name']
            tracks_data.append({
                "Artist name": artist_name,
                "Artist id": artist_id,
                "Track name": track_name,
                "Track popularity": track_popularity,
                "Genres": genres,
                "Followers": followers,
                "uri": uri 
            })
            print(f"Processed track {i+1}/{len(tracks)} by {artist_name} - Popularity: {track_popularity}")
        except Exception as e:
            print(f"Error processing track {i+1}: {e}")

    # Sort tracks_data by artist popularity and followers before creating DataFrame
    sorted_tracks_data = sorted(tracks_data, key=lambda x: x["Track popularity"], reverse=True)
    df_tracks_sorted = pd.DataFrame(sorted_tracks_data)

    print("Tracks sorted by artist popularity and followers")

    try:
        user_id = client.current_user()["id"]
        print(f"User ID: {user_id}")

        original_playlist = client.playlist(playlist_id)
        original_name = original_playlist['name']
        new_playlist_name = original_name + " - sorted by track popularity"
        new_playlist = client.user_playlist_create(user_id, new_playlist_name, public=False)
        new_playlist_id = new_playlist["id"]
        track_uris = df_tracks_sorted["uri"].tolist() 
        print(f"Creating new playlist with ID: {new_playlist_id}")

        for i in range(0, len(track_uris), 100):
            client.playlist_add_items(new_playlist_id, track_uris[i:i+100])
            print(f"Added tracks {i+1}-{min(i+100, len(track_uris))} to new playlist")
    except Exception as e:
        print(f"Error during playlist creation or track addition: {e}")

    return df_tracks_sorted

# Authentication

In [4]:
#sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())
# This will authenticate you on your web browser:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=os.getenv("SPOTIPY_CLIENT_ID"),
                                               client_secret=os.getenv("SPOTIPY_CLIENT_SECRET"),
                                               redirect_uri=os.getenv("SPOTIPY_REDIRECT_URI"),
                                               scope="playlist-modify-public playlist-modify-private"))

# Examples

Sorting a playlist by artist popularity:

In [4]:
# Italo-Disco TEST
# playlist_url = "https://open.spotify.com/playlist/2qajcUJ7x242tTuBpEKUKx"
# Dogs of TikTok/Instagram/YouTube TEST
playlist_url = "https://open.spotify.com/playlist/5XyfSYaCpPMmyTIynnq268"
# playlist_url = input("Please enter the playlist URL: ")
df_sorted = sort_playlist_by_artist_popularity(sp, playlist_url)
display(df_sorted)

Starting sort_playlist_by_artist_popularity
Playlist ID: 5XyfSYaCpPMmyTIynnq268
Tracks fetched: 42
Processed track 1/42 by Richard Strauss - Popularity: 52, Followers: 260534
Processed track 2/42 by Baha Men - Popularity: 51, Followers: 54309
Processed track 3/42 by Michael Jackson - Popularity: 84, Followers: 32438905
Processed track 4/42 by Kanye West - Popularity: 93, Followers: 27059637
Processed track 5/42 by Chance the Rapper - Popularity: 74, Followers: 6018968
Processed track 6/42 by Snoop Dogg - Popularity: 83, Followers: 12434839
Processed track 7/42 by Vanilla Ice - Popularity: 57, Followers: 530476
Processed track 8/42 by Elvis Presley - Popularity: 79, Followers: 9835074
Processed track 9/42 by The Proclaimers - Popularity: 63, Followers: 335390
Processed track 10/42 by Modest Mouse - Popularity: 64, Followers: 1485049
Processed track 11/42 by Weezer - Popularity: 75, Followers: 3775891
Processed track 12/42 by Fountains Of Wayne - Popularity: 58, Followers: 192612
Process

,Artist name,Artist id,Track name,Artist popularity,Genres,Followers,uri
0,Kanye West,5K4W6rqBFWDnAN6FQUkS6x,All Of The Lights,93,"[chicago rap, hip hop, rap]",27059637,spotify:track:22L7bfCiAkJo5xGSQgmiIO
1,Kanye West,5K4W6rqBFWDnAN6FQUkS6x,Runaway,93,"[chicago rap, hip hop, rap]",27059637,spotify:track:3DK6m7It6Pw857FcQftMds
2,Michael Jackson,3fMbdgg4jU18AjLCKBhRSm,Smooth Criminal - 2012 Remaster,84,"[r&b, soul]",32438905,spotify:track:2bCQHF9gdG5BNDVuEIEnNk
3,Red Hot Chili Peppers,0L8ExT028jH3ddEcZwqJJ5,Snow (Hey Oh),84,"[alternative rock, funk metal, funk rock, perm...",21284806,spotify:track:2aibwv5hGXSgw7Yru8IYTO
4,Snoop Dogg,7hJcb9fa4alzcOq3EaNPoG,Who Am I (What’s My Name)?,83,"[g funk, gangster rap, hip hop, pop rap, rap, ...",12434839,spotify:track:0eO8MW9YSTK3CjdaTYKlhF
5,Elvis Presley,43ZHCT0cAZBISjO8DG9PnE,Hound Dog,79,"[rock-and-roll, rockabilly]",9835074,spotify:track:64Ny7djQ6rNJspquof2KoX
6,Florence + The Machine,1moxjboGR7GNWYIMWsRjgG,Dog Days Are Over,78,"[baroque pop, modern rock, neo mellow, uk alte...",6825111,spotify:track:456WNXWhDwYOSf5SpTuqxd
7,Weezer,3jOstUTkEu2JkjvRdBA5Gu,Across The Sea,75,"[alternative rock, modern power pop, modern ro...",3775891,spotify:track:7JJPh9eOiSr8zYlBjcu2w0
8,Weezer,3jOstUTkEu2JkjvRdBA5Gu,Island In The Sun,75,"[alternative rock, modern power pop, modern ro...",3775891,spotify:track:2MLHyLy5z5l5YRp7momlgw
9,Chance the Rapper,1anyVhU62p31KFi8MEzkbf,All Night (feat. Knox Fortune),74,"[chicago rap, conscious hip hop, hip hop, pop ...",6018968,spotify:track:60xaS8mYBKUW4VQQ666N0T


Sorting by track popularity:

In [11]:
# Dogs of TikTok/Instagram/YouTube TEST
playlist_url = "https://open.spotify.com/playlist/5XyfSYaCpPMmyTIynnq268"
# playlist_url = input("Please enter the playlist URL: ")
df_sorted = sort_playlist_by_track_popularity(sp, playlist_url)
display(df_sorted)

Starting sort_playlist_by_track_popularity
Playlist ID: 5XyfSYaCpPMmyTIynnq268
Tracks fetched: 42
Processed track 1/42 by Richard Strauss - Popularity: 48
Processed track 2/42 by Baha Men - Popularity: 65
Processed track 3/42 by Michael Jackson - Popularity: 74
Processed track 4/42 by Kanye West - Popularity: 78
Processed track 5/42 by Chance the Rapper - Popularity: 65
Processed track 6/42 by Snoop Dogg - Popularity: 70
Processed track 7/42 by Vanilla Ice - Popularity: 70
Processed track 8/42 by Elvis Presley - Popularity: 67
Processed track 9/42 by The Proclaimers - Popularity: 75
Processed track 10/42 by Modest Mouse - Popularity: 56
Processed track 11/42 by Weezer - Popularity: 47
Processed track 12/42 by Fountains Of Wayne - Popularity: 47
Processed track 13/42 by Florence + The Machine - Popularity: 79
Processed track 14/42 by Sonny & Cher - Popularity: 64
Processed track 15/42 by Echo & the Bunnymen - Popularity: 61
Processed track 16/42 by The Libertines - Popularity: 61
Proces

,Artist name,Artist id,Track name,Track popularity,Genres,Followers,uri
0,Kanye West,5K4W6rqBFWDnAN6FQUkS6x,Runaway,81,"[chicago rap, hip hop, rap]",27059637,spotify:track:3DK6m7It6Pw857FcQftMds
1,Florence + The Machine,1moxjboGR7GNWYIMWsRjgG,Dog Days Are Over,79,"[baroque pop, modern rock, neo mellow, uk alte...",6825111,spotify:track:456WNXWhDwYOSf5SpTuqxd
2,Weezer,3jOstUTkEu2JkjvRdBA5Gu,Island In The Sun,79,"[alternative rock, modern power pop, modern ro...",3775891,spotify:track:2MLHyLy5z5l5YRp7momlgw
3,Kanye West,5K4W6rqBFWDnAN6FQUkS6x,All Of The Lights,78,"[chicago rap, hip hop, rap]",27059637,spotify:track:22L7bfCiAkJo5xGSQgmiIO
4,Red Hot Chili Peppers,0L8ExT028jH3ddEcZwqJJ5,Snow (Hey Oh),78,"[alternative rock, funk metal, funk rock, perm...",21284806,spotify:track:2aibwv5hGXSgw7Yru8IYTO
5,The Proclaimers,1A92IAcd7A6npCA33oGM5i,I'm Gonna Be (500 Miles),75,"[celtic rock, scottish rock]",335390,spotify:track:67iAlVNDDdddxqSD2EZhFs
6,Huey Lewis & The News,7A9yZMTrFZcgEWAX2kBfK6,The Power Of Love,75,"[album rock, classic rock, hard rock, mellow g...",1630485,spotify:track:2olVm1lHicpveMAo4AUDRB
7,Michael Jackson,3fMbdgg4jU18AjLCKBhRSm,Smooth Criminal - 2012 Remaster,74,"[r&b, soul]",32438905,spotify:track:2bCQHF9gdG5BNDVuEIEnNk
8,Snoop Dogg,7hJcb9fa4alzcOq3EaNPoG,Who Am I (What’s My Name)?,70,"[g funk, gangster rap, hip hop, pop rap, rap, ...",12434839,spotify:track:0eO8MW9YSTK3CjdaTYKlhF
9,Vanilla Ice,7GXXMm3DB1VswVcuGyInUd,Ice Ice Baby,70,"[miami bass, miami hip hop]",530476,spotify:track:3XVozq1aeqsJwpXrEZrDJ9


Bands and artists I've seen live:

In [6]:
seen_live = sort_playlist_by_artist_popularity(sp, "https://open.spotify.com/playlist/5ZAVOxwjVynsdh4FXypiI7")
display(seen_live)

Starting sort_playlist_by_artist_popularity
Playlist ID: 5ZAVOxwjVynsdh4FXypiI7
Tracks fetched: 476
Processed track 1/476 by The Weeknd - Popularity: 96, Followers: 92131762
Processed track 2/476 by Kendrick Lamar - Popularity: 92, Followers: 33275466
Processed track 3/476 by Myke Towers - Popularity: 89, Followers: 16626010
Processed track 4/476 by Imagine Dragons - Popularity: 89, Followers: 54761225
Processed track 5/476 by Arctic Monkeys - Popularity: 87, Followers: 26765393
Processed track 6/476 by Green Day - Popularity: 83, Followers: 15515258
Processed track 7/476 by Manuel Turizo - Popularity: 84, Followers: 14032248
Processed track 8/476 by Radiohead - Popularity: 83, Followers: 10612689
Processed track 9/476 by ROSALÍA - Popularity: 83, Followers: 8338703
Processed track 10/476 by System Of A Down - Popularity: 82, Followers: 11401599
Processed track 11/476 by Daft Punk - Popularity: 82, Followers: 10172960
Processed track 12/476 by Tame Impala - Popularity: 81, Followers: 8

,Artist name,Artist id,Track name,Artist popularity,Genres,Followers,uri
0,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,Wicked Games,96,"[canadian contemporary r&b, canadian pop, pop]",92131762,spotify:track:7bGF2hNOP6SROacsxo1fCk
1,Kendrick Lamar,2YZyLoL8N0Wb9xBt1NhZWg,"Bitch, Don’t Kill My Vibe",92,"[conscious hip hop, hip hop, rap, west coast rap]",33275466,spotify:track:5h5tBFnbcVioFXiOixTn6E
2,Imagine Dragons,53XhwfbYqKCa1cC15pYq2q,Radioactive,89,"[modern rock, pop, rock]",54761225,spotify:track:62yJjFtgkhUrXktIoSjgP2
3,Myke Towers,7iK8PXO48WeuP03g8YR51W,LALA,89,"[reggaeton, trap latino, urbano latino]",16626010,spotify:track:7ABLbnD53cQK00mhcaOUVG
4,Arctic Monkeys,7Ln80lUS6He07XvHI8qqHH,The View From The Afternoon,87,"[garage rock, modern rock, permanent wave, roc...",26765393,spotify:track:4fCpHhES8IExESfkZmRQQ3
...,...,...,...,...,...,...,...
471,Jacob Singer,4fW9zW7nMHEYegoVFpPDzc,Kalypso,1,[],153,spotify:track:1l0gQSMSvGmbUgwCQWGzSm
472,The Casters,512iC7gh9ITAZ0mbmYM5S1,Once We Were,0,[],681,spotify:track:6CbjUsRddlZi5nw3Emooci
473,Zenit con Ari y Frank-T,7pC599SwpMRtmnkpRssFoU,Suelo Soñar,0,[rap underground espanol],392,spotify:track:7BTOJ8cvgZuYslN8TCWfg6
474,Biscuit Miller,75cfGBT6F2Jy1z3NF2eAbA,She Likes to Boogie,0,[],289,spotify:track:33JAedQLu0Ne9DFGDIJcrV


### Printing my list of bands and artists I've seen live, ranked by popularity:

In [7]:
pd.set_option('display.max_rows', None)
seen_live.index += 1 
display(seen_live[['Artist name', 'Artist popularity', 'Followers']])
pd.reset_option('display.max_rows')

,Artist name,Artist popularity,Followers
1,The Weeknd,96,92131762
2,Kendrick Lamar,92,33275466
3,Imagine Dragons,89,54761225
4,Myke Towers,89,16626010
5,Arctic Monkeys,87,26765393
6,Manuel Turizo,84,14032248
7,Green Day,83,15515258
8,Radiohead,83,10612689
9,ROSALÍA,83,8338703
10,System Of A Down,82,11401599
